---
layout: article
title: How much money do we spend in a lifetime?
custom_css: article.css
include_mathjax: true
---

In [ ]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.ticker import FormatStrFormatter
from scipy.optimize import minimize

# define watermark 
def add_watermark(ax, x, y):
    ax.text(ax.get_xlim()[0]+ x,
            ax.get_ylim()[0]+ y,
            "ladydragoncapital",
            alpha=0.3, fontsize=16)



## Introduction ##

Most people want to make money, but few think about why they need money and how much money they need. In this article we want to answer the following questions:

+ How much is the lifetime expense for an average person?
+ How different can the lifetime expense be in different countries with different levels of economic development?
+ How different can the lifetime expense be under different lifestyles?
+ How much money is still required under the minimum lifestyle?


## Three stages ##

The average life expectancy in America is 78 years old. Let's divide the life of an average person into three stages:

+ Childhood and early adulthood (0-23)
  
   The person was born healthily, and his parents could afford the average cost of living and insurance. He went to public schools, and finished public college at the age of 23. 
  
+ Adulthood (24-60)
  
  He started working after college, and continued his life at the average cost of living, rented an apartment, and used public transportation. To simplify the initial calculation, we use renting as the only option for housing, and public transportation for transportation. He paid health and home insurances.
  
+ Retirement (61-78)
  
  After retiring at 60, his expenses included average cost of living, rent, public transportation and insurances.

## Calculation of expenses ##

The three stages will be calculated separately:

**Childhood and early adulthood (0-23)**: 

$$
E_{childhood} =  \sum_{i=1}^{23} (C_{cl} + C_{chin}) (1 + f)^{i-1} + C_{education}
$$
    
Where:

+ $E_{childhood}$ is the total expense from age 0 to 23
+ $C_{cl}$ is the annual cost of living for children, which is 80% of the cost for adults
+ $C_{chin}$ is the annual health insurance payment for children, which is 80% of the cost for adults
+ $C_{education}$ is the total tuition for college
+ $f$ is the average inflation rate. 

**Adulthood (24-60)**: 


$$
E_{adulthood} = \sum_{i=24}^{60} (C_{al} + C_{arent} + C_{atr} + C_{ahin} + C_{arin}) (1+f)^{i-24} 
$$

Where:

+ $E_{adulthood}$ is the total expense from age 24 to 60
+ $C_{al}$ is the annual cost of living for adults
+ $C_{arent}$ is the annual rent for adults
+ $C_{atr}$ is the annual cost for public transportation for adults
+ $C_{ahin}$ is the annual health insurance payment for adults
+ $C_{arin}$ is the annual renting home insurance 
+ $f$ is the average inflation rate. 

Considering the inflation rate and its influence on cost of living and insurance, we have the following relationship between the cost of living and health insurance of childhood and adulthood:

$$
C_{al} = \frac{C_{cl}}{0.8}(1+f)^{23}
$$

$$
C_{ahin} = \frac{C_{chin}}{0.8}(1+f)^{23}
$$

**Retirement (61-78)**: 

$$
E_{retirement} = \sum_{i=61}^{78} (C_{rl} + C_{rrent} + C_{rtr} + C_{rhin} + C_{rrin} ) (1+r)^{i-61}
$$

Where:

+ $E_{retirement}$ is the total expense from age 61 to 78
+ $C_{rl}$ is the annual cost of living for the retired people
+ $C_{rrent}$ is the annual rent for the retired
+ $C_{rtr}$ is the annual cost for public transportation for the retired
+ $C_{rhin}$ is the annual health insurance payment for the retired
+ $C_{rrin}$ is the annual renting home insurance 
+ $f$ is the average inflation rate

Considering the inflation rate and its influence on cost of living and insurance, we have the following relationship between the cost of living, rent, transportation, and insurance of childhood and adulthood:

$$
C_{rl} = C_{al}(1+f)^{36}
$$

$$
C_{rrent} = C_{arent}(1+f)^{36}
$$

$$
C_{rtr} = C_{atr}(1+f)^{36}
$$

$$
C_{rhin} = C_{ahin}(1+f)^{36}
$$

$$
C_{rrin} = C_{arin}(1+f)^{36}
$$

## The lifetime expense in different countries ##

We calculated the average lifetime expenses in four countries:

+ the United States
+ China
+ Ecuador
+ Argentina

In our calculation, we used the following data ([Numero.com](https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States)):

+ the United States:
    + average public college tuition: $11260/year
    + average cost of living for an adult: $1171.1/month
    + average health insurance: $8951/year
    + average rent: 1436.84/month
    + average home insurance: $240/year
    + average cost of public transportation: $65/month
      
+ China:
    + average public college tuition: $823.05/year
    + average cost of living for an adult: $521.2/month
    + average health insurance: $54.85/year
    + average rent: $268.03/month
    + average home insurance: 0
    + average cost of public transportation: $24.63/month

+ Ecuador:
    + average public college tuition: 0
    + average cost of living for an adult: $532.5/month
    + average health insurance: $552/year
    + average rent: $273.93/month
    + average home insurance: 0
    + average cost of public transportation: $25/month

+ Argentina:
    + average public college tuition: 0
    + average cost of living for an adult: $678.1/month
    + average health insurance: 0
    + average rent: 334.73/month
    + average home insurance: 0
    + average cost of public transportation: $21/month

Note: home insurance is only common in the US, often required by landlords or banks.

In [ ]:
# define total expense
def total_expense(e, cl, hin, rin, r, t, f, n):
    """
    calculate total expense from age 0 to death with annual inflation.
    Args:
    cl (float): Annual cost of living at age 0.
    hin (float): Annual health insurance at age 0.
    rin (float): Anuual rental home insurance at age 24.
    r (float): annual rent at age 24.
    t (float): Annual transportation fee (public transportation) at age 24
    f (float): inflation rate (> 0)
    n (int): age
    """
    if 0 <= n <= 23:
        child_annual = cl + hin
        total = child_annual * ((1 + f)**(n + 1) - 1) / f + 4 * e
    elif 24 <= n <= 59:
        child_annual = cl + hin
        child_total = child_annual * ((1 + f)**24 - 1) / f + 4 * e
        cl_adt = cl/0.8*((1+f)**23)
        hin_adt = hin/0.8*((1+f)**23)
        adult_annual = cl_adt + hin_adt + r + t + rin
        adult_total = adult_annual * ((1 + f)**(n - 22) - 1) / f
        total = child_total + adult_total
    elif n >= 60:
        child_annual = cl + hin
        child_total = child_annual * ((1 + f)**24 - 1) / f + 4 * e
        cl_adt = cl/0.8*((1+f)**23)
        hin_adt = hin/0.8*((1+f)**23)
        adult_annual = cl_adt + hin_adt + r + t + rin
        adult_total = adult_annual * ((1 + f)**38 - 1) / f
        cl_rt = cl_adt*((1+f)**36)
        hin_rt = hin_adt*((1+f)**36)
        r_rt = r*((1+f)**36)
        rin_rt = rin*((1+f)**36)
        t_rt = t*((1+f)**36)
        retire_annual = cl_rt + hin_rt + r_rt + rin_rt + t_rt
        retire_total = retire_annual * ((1 + f)**(n - 60) - 1) / f
        total = child_total + adult_total + retire_total
    return total
        

In [ ]:
# visualize the lifetime expense from 0 -80
fig_count = 1
# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# set values for the US
e = 11260 # avg. public in-state college tuition 11260 (2024)
cl = 1171.1 * 12 * 0.8 # cost of living is 1171.1/month (2024)
hin = 8951 * 0.8 # health insurance is 8951/year (2024)
rin = 240 # home insurance for rented apt 240/year (2024)
r = 1436.84 * 12 # rent for one-bedroom apt (2024)
t = 65 * 12 # public transportation 65/month
f = 0.02
n = 78

# set values for China
e_cn = 823.05 # average public college tuition 823.05
cl_cn = 521.2 * 12 * 0.8
hin_cn = 54.85
rin_cn = 0 # home insurance is under-developed at the moment
r_cn = 268.03
t_cn = 24.63 * 12
f_cn = 0.02
n_cn = 78

# set values for Ecuador
e_ed = 0 # public college tuition 0.
cl_ed = 523.5 * 12 * 0.8
hin_ed = 552 * 0.8
rin_ed = 0 # home insurance is under-developed at the moment
r_ed = 273.93
t_ed = 25 *12
f_ed = 0.02
n_ed = 78


# set values for Argentina
e_ar = 0 # public college tuition 0.
cl_ar = 678.1 * 12 * 0.8
hin_ar = 0 # public basic insurance is free
rin_ar = 0 # home insurance is under-developed at the moment
r_ar = 334.73
t_ar = 21 * 12
f_ar = 0.02
n_ar = 78


fig, ax = plt.subplots(figsize=(8, 6))


# generate p/ΔN pairs
ages = np.arange(0, 78, 1)
expenses = np.array([total_expense(e, cl, hin, rin, r, t, f, n) for n in ages])
expenses_cn = np.array([total_expense(e_cn, cl_cn, hin_cn, rin_cn, r_cn, t_cn, 
                                      f_cn, n_cn) for n_cn in ages])
expenses_ed = np.array([total_expense(e_ed, cl_ed, hin_ed, rin_ed, r_ed, t_ed, 
                                      f_ed, n_ed) for n_ed in ages])
expenses_ar = np.array([total_expense(e_ar, cl_ar, hin_ar, rin_ar, r_ar, t_ar, 
                                      f_ar, n_ar) for n_ar in ages])



# plotting the data
plt.plot(ages, expenses, label='United States')
plt.plot(ages, expenses_cn, label='China')
plt.plot(ages, expenses_ed, label='Ecuador')
plt.plot(ages, expenses_ar, label='Argentina')

# add watermark
add_watermark(ax, 3, 1600000)

# adding labels and title
plt.xlabel("Age")
plt.ylabel("Total Expense (Dollar)")

# set title
plt.suptitle(
    f"Figure {fig_count}. Lifetime expense growth curve from age 0-80", y=0.0001, fontsize=10
)
fig_count += 1


# displaying the plot
plt.legend()
plt.show()

Figure 1 shows, if a person is born in 2024 and lives till the age of 78, how the total expense grows with time in those four countries. Due to the high cost of living and health insurance, the lifetime expense grows much faster in the US. Also as shown in Figure 2, the total expense will be 6.04 million dollars in the US, much higher than the total expense in other countries: 1.21 million in China, 1.30 million in Ecuador, and 1.54 million in Argentina.

In [ ]:
# visualize total expense
# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

us = total_expense(e, cl, hin, rin, r, t, f, 78)
cn = total_expense(e_cn, cl_cn, hin_cn, rin_cn, r_cn, t_cn, f_cn, 78)
ed = total_expense(e_ed, cl_ed, hin_ed, rin_ed, r_ed, t_ed, f_ed, 78)
ar = total_expense(e_ar, cl_ar, hin_ar, rin_ar, r_ar, t_ar, f_ar, 78)


# data for bar plot
countries = ['China', 'Ecuador', 'Argentina', 'US']
expenses = [cn, ed, ar, us]

fig, ax = plt.subplots(figsize=(8, 6))
bars = plt.bar(countries, expenses, width=0.6)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}',
             ha='center', va='bottom', fontsize=12)

# add watermark
add_watermark(ax, 0, 1600000)

# adding labels and title
plt.xlabel("Country")
plt.ylabel("Total Expense (Dollar)")

# set title
plt.suptitle(
    f"Figure {fig_count}. Lifetime expense from age 0-78", y=0.0001, fontsize=10
)
fig_count += 1


# displaying the plot
#plt.legend()
plt.show()

## Advanced lifestyle ##

In reality, on top of the simplified calculation above, many people are not satisfied with renting and public transportation, which will result in other major purchases, such a house and a car. Considering these major costs, while the expense during childhood remains the same as the basic lifestyle, the expense during adulthood and retirement can be calculated as following:

**Adulthood (24-60)**: 

$$
E_{adulthood} = \sum_{i=24}^{60} (C_{al} + C_{insurances}) (1+f)^{i-24} + C_{house} + C_{cars}
$$

Where:

+ $C_{al}$ is the average cost of living for an adult
+ $C_{insurances}$ includes health, home, and car insurances
+ $C_{house}$ is the average cost for a single-family house
+ $C_{cars}$ is the cost for cars, including three cars plus inflation since the average lifespan of a car is 15 years.

**Retirement (61-78)**: 

$$
E_{retirement} = \sum_{i=61}^{78} (C_{rl} + C_{insurances}) (1+r)^{i-61}
$$

Where:

+ $C_{rl}$ is the average cost of living for a retired person
+ $C_{insurances}$ includes health, home, and car insurances

If a house and a car is fully purchased before retirement, the major expenses during this period will only include cost of living and insurances (health, home, and car). 

In Figure 3, we compared the basic and advanced lifestyle in the US. In the calculation of the basic lifetime expense, we used the same data as Figure 2. In the calculation of the advanced lifetime expense, we used the following data:

+ average public college tuition: $11260/year
+ average cost of living for an adult: $1171.1/month
+ average health insurance: $8951/year
+ average price for a single-family house: $420,000
+ average price for a Toyota Corolla Sedan (2024): $25,968
+ average home insurance: $2110/year
+ average car insurance: $2685/year

In [ ]:
# define total expense
def total_expense_advanced(e, cl, hein, hoin, cin, car, house, f, n):
    """
    calculate total expense from age 0 to death with annual inflation.
    Args:
    cl (float): Annual cost of living at age 0.
    hin (float): Annual health insurance at age 0.
    rin (float): Anuual rental home insurance at age 24.
    r (float): annual rent at age 24.
    t (float): Annual transportation fee (public transportation) at age 24
    f (float): inflation rate (> 0)
    n (int): age
    """
    if 0 <= n <= 23:
        child_annual = cl + hein
        total = child_annual * ((1 + f)**(n + 1) - 1) / f + 4 * e
    elif 24 <= n <= 59:
        child_annual = cl + hein
        child_total = child_annual * ((1 + f)**24 - 1) / f + 4 * e
        cl_adt = cl/0.8*((1+f)**23)
        hein_adt = hin/0.8*((1+f)**23)
        adult_annual = cl_adt + hein_adt + hoin + cin
        cars = car + car*(1+f)**15 + car*(1+f)**30 
        adult_total = adult_annual * ((1 + f)**(n - 22) - 1) / f + cars + house
        total = child_total + adult_total
    elif n >= 60:
        child_annual = cl + hein
        child_total = child_annual * ((1 + f)**24 - 1) / f + 4 * e
        cl_adt = cl/0.8*((1+f)**23)
        hein_adt = hin/0.8*((1+f)**23)
        adult_annual = cl_adt + hein_adt + hoin + cin
        cars = car + car*(1+f)**15 + car*(1+f)**30 
        adult_total = adult_annual * ((1 + f)**(n - 22) - 1) / f + cars + house
        cl_rt = cl_adt*((1+f)**36)
        hein_rt = hein_adt*((1+f)**36)
        hoin_rt = hoin*((1+f)**36)
        cin_rt = cin*((1+f)**36)
        retire_annual = cl_rt + hein_rt + hoin_rt + cin_rt
        retire_total = retire_annual * ((1 + f)**(n - 60) - 1) / f
        total = child_total + adult_total + retire_total
    return total
        

In [ ]:
# visualize basic and advanced lifestyle
# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# set values for basic lifestyle the US
e = 11260 # avg. public in-state college tuition 11260 (2024)
cl = 1171.1 * 12 * 0.8 # cost of living is 1171.1/month (2024)
hin = 8951 * 0.8 # health insurance is 8951/year (2024)
rin = 240 # home insurance for rented apt 240/year (2024)
r = 1436.84 * 12 # rent for one-bedroom apt (2024)
t = 65 * 12 # public transportation 65/month
f = 0.02
n = 78

# set values for advanced lifestyle in the US
e_ad = 11260
cl_ad = 1171.1 * 12 * 0.8
hein_ad = 8951 * 0.8
hoin_ad = 2110 # for 300,000 in dwelling
cin_ad = 2685 # for full coverage
house = 420000
car = 25967.99 # price for Toyota Corolla Sedan 2024
f_ad = 0.02
n_ad = 78

fig, ax = plt.subplots(figsize=(8, 6))


# calculate remaining expenses
expenses_bsc = total_expense(e, cl, hin, rin, r, t, f, 78)
expenses_ad = total_expense_advanced(e_ad, cl_ad, hein_ad, hoin_ad, cin_ad, car, house, f, n)

# data for bar plot
styles = ['Basic', 'Advanced']
expenses = [expenses_bsc, expenses_ad]
bars = plt.bar(styles, expenses, width=0.3)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}',
             ha='center', va='bottom', fontsize=12)

# add watermark
add_watermark(ax, 0, 1600000)

# adding labels and title
plt.xlabel("Lifestyle")
plt.ylabel("Total Expense (Dollar)")

# set title
plt.suptitle(
    f"Figure {fig_count}. Lifetime expense for difference lifestyles in the US", y=0.0001, fontsize=10
)
plt.xlim([-1,2])
fig_count += 1

# displaying the plot
plt.show()

In Figure 3, under the more advanced lifestyle that involves house and multiple car purchases in America, the total lifetime expense will increase from 6.05 million dollars to 7.10 dollars (17%).

Comparing Figure 2 and 3, we can see the influence of changing lifestyle on the total expense seems to be lower than changing country. When a person moves from the US to Argentina and maintain the basic lifestyle, the total expense can decrease 75% (6.05 million to 1.54 million), while a person lower the standard of life quality from advanced to basic within the US can only decrease the total expense by 15% (7.10 million to 6.05 million). Therefore, if your goal is to significantly lower the total expense, moving to other countries with a lower cost of living can be more effective than lowering the standard of life within the same country.

## Remaining expense ##

Sam is very good lowering his expenses. After a lot of hard effort, at the age of 35, he was able to lower his expense to the following level:

+ cost of living: $70/month
  
+ health insurance: $54.85/year
  
+ rent: $30/month
  
+ home insurance: 0
  
+ cost of public transportation: $20/month

If Sam continues this lifestyle, how much money will he still need before death (78 years old)?

In [ ]:
# define total expense
def remaining_expense(n, f, l, al, hin, rin, r, t):
    """
    calculate total expense from age 0 to death with annual inflation.
    Args:
    n (int): age (>23)
    f (float): inflation rate (> 0)
    l (int): lifespan
    al (float): Annual basic expense at age 24-59, 60+, respectively.
    hin (float): Annual medical expense at age 24-59, 60+, respectively.
    r (float): annual rent at age 24.
    t (float): Annual transportation fee (public transportation) at age 24
    """
    if 0 <= n <= 23:
        raise ValueError("n must be higher than 23.")
    # pre-60 expense
    t1 = max(0, 60 - n)
    adult_cost = al + hin + rin + r + t 
    if f == 0:
        adult_total = adult_cost * t1
    else:
        adult_total = adult_cost * ((1 + f)**t1 - 1) / f

    # Post-60 period
    t2 = l - 60
    retire_cost = (al + hin + rin + r + t)*((1 + f)**t1) # adjusted for inflation 
    if f == 0:
        retire_total = retire_cost * t2
    else:
        retire_total = retire_cost * ((1 + f)**t2 - 1) / f
    return adult_total + retire_total

In [ ]:
# visualize remaining expense
# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# set values for Sam
n = 35
f = 0.02
l = 78
al = 100 * 12 
hin = 54.85
rin = 0 
r = 30
t = 200 # public transportation

# set values for China
n_cn = 35
f_cn = 0.02
l_cn = 78
al_cn = 522.0 * 12 
hin_cn = 54.85
rin_cn = 0 
r_cn = 268.03
t_cn = 24.63 * 12# public transportation

# set values for Ecuador
n_ed = 35
f_ed = 0.02
l_ed = 78
al_ed = 525.1 * 12 
hin_ed = 552
rin_ed = 0 
r_ed = 273.93
t_ed = 25 * 12 # public transportation

# set values for Argentina
n_ar = 35
f_ar = 0.02
l_ar = 78
al_ar = 678.1 * 12 
hin_ar = 0
rin_ar = 0 
r_ar = 334.73
t_ar = 21 *12

# set values for the US
n_us = 35
f_us = 0.02
l_us = 78
al_us = 1181.3 * 12 
hin_us = 8951
rin_us = 240
r_us = 1436.84 *12
t_us = 65 * 12

fig, ax = plt.subplots(figsize=(8, 6))


# calculate remaining expenses
expenses_sam = remaining_expense(n, f, l, al, hin, rin, r, t)
expenses_cn = remaining_expense(n_cn, f_cn, l_cn, al_cn, hin_cn, rin_cn, r_cn, t_cn)
expenses_ed = remaining_expense(n_ed, f_ed, l_ed, al_ed, hin_ed, rin_ed, r_ed, t_ed)
expenses_ar = remaining_expense(n_ar, f_ar, l_ar, al_ar, hin_ar, rin_ar, r_ar, t_ar)
expenses_us = remaining_expense(n_us, f_us, l_us, al_us, hin_us, rin_us, r_us, t_us)

# data for bar plot
countries = ['Sam', 'China', 'Ecuador', 'Argentina', 'US']
expenses = [expenses_sam, expenses_cn, expenses_ed, expenses_ar, expenses_us]
bars = plt.bar(countries, expenses, width=0.6)

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}',
             ha='center', va='bottom', fontsize=12)

# add watermark
add_watermark(ax, 0, 1000000)

# adding labels and title
plt.xlabel("Country")
plt.ylabel("Total Expense (Dollar)")

# set title
plt.suptitle(
    f"Figure {fig_count}. Lifetime expense from age 35-78", y=0.0001, fontsize=10
)
fig_count += 1


# displaying the plot
#plt.legend()
plt.show()

Figure 4 shows that under Sam's current lifestyle, the remaining expense of his life is 99,721.74 USD. Meanwhile if Sam moves to those four countries and lives under the basic lifestyle there, the remaining expense of his life will be 462,221.01 USD in China, 498,892.1 USD in Ecuador, 585,894.5 USD in Argentina, and 2,779,641.72 in the US.

Even though Sam did a lot of work to lower his expense, as shown in Figure 4, with the minimum lifestyle, Sam's remaining expense can still grow into a large number due to its compounding effect throughout decades. Therefore, while trying to lower the expense, a reasonable amount of effort needs to be invested into growing wealth to compensate the expense.

## Conclusion ##

From our calculation, we are able to acquire the following understanding:

+ The lifetime expense for the average people can vary significantly due to the country they choose to live in.
  
+ The lifetime expense can vary due to the lifestyle, such as renting vs. owning a home; public vs. private transportation.

+ In order to reduce the lifetime expense, lowering the standard of quality of life might be less effective than moving to another country.
  
+ Even with the minimum lifestyle, the total expense can still grow into a large number due to its compounding effect. While trying to lower the expense, a reasonable amount of effort needs to be invested into growing wealth.

## References ##

+ https://www.cdc.gov/nchs/fastats/life-expectancy.htm
+ https://baijiahao.baidu.com/s?id=1824533769859452187&wfr=spider&for=pc
+ https://www.numbeo.com/cost-of-living/country_result.jsp?country=United+States